In [1]:
# import libraries
import urllib.request
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import urllib.request

# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
#print(soup.prettify())

In [2]:
# extracting the raw table inside that webpage
from bs4 import BeautifulSoup

table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [3]:
#append columns here neighbourhood is not assigned and the borough is the same
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
                
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 181


In [4]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
#df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M9W,Etobicoke,Northwest
1,M3Y,Not assigned,Not assigned
2,M7P,Not assigned,Not assigned
3,M5S,Downtown Toronto,"Harbord, University of Toronto"
4,M3J,North York,"Northwood Park, York University"
5,M2H,North York,Hillcrest Village
6,M3S,Not assigned,Not assigned
7,M7G,Not assigned,Not assigned
8,M8E,Not assigned,Not assigned
9,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."


In [5]:
#deleting data where Borough = 'Not Assigned'
df_toronto = df_toronto[df_toronto.Borough != 'Not assigned']

In [6]:
df_toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M9W,Etobicoke,Northwest
3,M5S,Downtown Toronto,"Harbord, University of Toronto"
4,M3J,North York,"Northwood Park, York University"
5,M2H,North York,Hillcrest Village
9,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
12,M6A,North York,"Lawrence Heights, Lawrence Manor"
13,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
16,M2N,North York,Willowdale South
17,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
20,M2R,North York,Willowdale West


In [7]:
#import coordinates data
path='Geospatial_Coordinates.csv'

df = pd.read_csv(path)



In [8]:
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
#rename postal code feild to match df_toronto df
df = df.rename(index=str, columns={"Postal Code": "Postcode"})

In [27]:
df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [31]:
#merging two dataframes on postcode into a new data
df_ton = pd.merge(left=df_toronto, right=df, on="Postcode", how="left")

In [33]:
df_ton.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049
2,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
3,M2H,North York,Hillcrest Village,43.803762,-79.363452
4,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
